In [1]:
from litequeue import LiteQueue, MessageStatus
import sqlite3

import gc

from string import ascii_lowercase, printable
from random import choice


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [2]:
print(sqlite3.sqlite_version)

'3.40.0'

## Test `retry` and `failed`

In [1]:
from litequeue import LiteQueue

q = LiteQueue(":memory:", maxsize=50)

In [2]:
q.put("hiello")

task = q.pop()

task

{'message': 'hiello',
 'message_id': 'b4eda2e0cbe6af7ee66134d189bbb2b0',
 'status': 1,
 'in_time': 1676224931,
 'lock_time': 1676224931,
 'done_time': None}

In [4]:
[dict(x) for x in q.list_locked(2)]

[{'message': 'hiello',
  'message_id': 'b4eda2e0cbe6af7ee66134d189bbb2b0',
  'status': 1,
  'in_time': 1676224931,
  'lock_time': 1676224931,
  'done_time': None}]

## Benchmarks

In [31]:
import gc

In-memory SQL queue

In [32]:
q = LiteQueue(":memory:", maxsize=None)

In [33]:
gc.collect()

6

In [34]:
%%timeit -n10000 -r7

q.put(random_string(20))

18.2 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [35]:
q.qsize()

70000

Standard python queue.

In [36]:
from queue import Queue

In [37]:
q = Queue()

In [38]:
gc.collect()

6

In [39]:
%%timeit -n10000 -r7

q.put(random_string(20))

9.23 µs ± 460 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Persistent SQL queue

In [40]:
q = LiteQueue("test.queue", maxsize=None)

In [41]:
gc.collect()

3

In [42]:
%%timeit -n10000 -r7

q.put(random_string(20))

87 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [43]:
!du -sh test.queue*

8.4M	test.queue
32K	test.queue-shm
4.0M	test.queue-wal


In [44]:
!rm test.queue*

In [45]:
assert q.conn.isolation_level is None

Creating and removing tasks

In [46]:
q = Queue()

In [47]:
gc.collect()

6

In [48]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

q.get()

q.task_done()

11.4 µs ± 311 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [49]:
q = LiteQueue(":memory:", maxsize=None)

In [50]:
gc.collect()

3

In [51]:
%%timeit -n10000 -r7

tid = random_string(20)

q.put(tid)

task = q.pop()

q.done(task["message_id"])

51 µs ± 946 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


**RETURNING vs. TRANSACTION**

RETURNING

In [52]:
q = LiteQueue("pop_bench.db", maxsize=None)

q.pop = q._pop_returning

gc.collect()

6

In [53]:
for _ in range(10000):
    tid = random_string(60)

    q.put(tid)

In [54]:
%%time


for _ in range(8000):
    task = q.pop()

CPU times: user 304 ms, sys: 152 ms, total: 455 ms
Wall time: 555 ms


In [55]:
!rm pop_bench.db*

In [56]:
q = LiteQueue("pop_bench.db", maxsize=None)

q.pop = q._pop_transaction

gc.collect()

3

In [57]:
for _ in range(10000):
    tid = random_string(60)

    q.put(tid)

In [58]:
%%time


for _ in range(8000):
    task = q.pop()

CPU times: user 5.27 s, sys: 162 ms, total: 5.43 s
Wall time: 5.53 s


In [59]:
!rm pop_bench.db*